In [51]:
import pickle
mol = "h4"
with open("./planted_solution/" + mol + ".pkl", "rb") as f:
    table = pickle.load(f)
with open("./planted_solution/" + mol + " Hamiltonians.pkl", "rb") as f:
    dic = pickle.load(f)  
print(dic)

{'[[0, 1], [2, 3], [4, 5], [6, 7]]': array([-1.36029582,  0.49473265,  0.49473265, -0.16941026,  0.49473266,
       -0.41097147, -0.41097146, -0.49452889,  0.49473265, -0.41097147,
       -0.41097148, -0.49452887, -0.16941025, -0.4945289 , -0.49452887,
       -1.35980526, -0.83309736,  0.31304004,  0.31304005, -0.75365504,
        0.31304004, -0.92273598, -0.92273598, -0.01030323,  0.31304005,
       -0.92273598, -0.92273597, -0.01030323, -0.75365503, -0.01030323,
       -0.01030323, -0.67944054, -1.59719096,  0.57271339,  0.5727134 ,
       -0.07589833,  0.57271338, -0.32599873, -0.32599873, -0.29323355,
        0.57271338, -0.32599873, -0.32599872, -0.29323355, -0.07589833,
       -0.29323354, -0.29323357, -0.85481153, -1.07421019, -0.10217657,
       -0.10217658,  0.21434027, -0.10217657,  0.0863873 ,  0.0863873 ,
        0.12291051, -0.10217658,  0.08638732,  0.0863873 ,  0.1229105 ,
        0.21434029,  0.12291051,  0.1229105 , -1.37700107,  0.84320667,
        1.50926885,  0.2691

In [26]:
def orbitals_to_partitions(lis: list) -> str:
    return "CAS "+"-".join([str(len(i)) for i in lis])
for i in table[1:]:
    if i[3] < 1:
        print(orbitals_to_partitions(i[0]), end = " & ")
        print(" & ".join(str(round(k, 3)) for k in i[1:]), end = " \\\\\n")
#     if isinstance(i, list):
#         print()
# #         print("| ", end = "")
#         print(orbitals_to_partition(i), end = " & ")
#     else:
#         if i > 10:
#             print(round(i,3), end = " \\\\")
#         else: 
#             print(round(i,3), end = " & ")

CAS 4-5-5 & 2609.526 & 0.948 & 0.781 & 6447.158 \\
CAS 2-12 & 2367.738 & 0.86 & 0.422 & 6454.523 \\
CAS 4-4-6 & 2602.814 & 0.945 & 0.74 & 6438.205 \\
CAS 2-2-10 & 2508.529 & 0.911 & 0.589 & 6385.453 \\
CAS 4-10 & 2469.411 & 0.897 & 0.939 & 6144.44 \\


In [57]:
import json
import sys
sys.path.append("../")
import csa_utils as csau
import ferm_utils as feru
import openfermion as of

import numpy as np
for key in dic:
    k = json.loads(key)
    n = max([max(block) for block in k]) + 1
    break
print(n + 1)
for key in dic:
    k = json.loads(key)
    if k == [[0, 1], [2, 3], [4, 5, 6, 7]]:
        print(dic[key].shape)
        print(k)
        cur_tbt = csau.sum_cartans(dic[key], n, k, 1, complex=False)
        k = [[0, 1, 2, 3], [4, 5, 6, 7]]
        count = 0
        for i in range(3):
            sol = csau.csa(cur_tbt, k)
            decomp_cost = csau.ct_decomp_cost(sol.x, k, cur_tbt, alpha = 1, complex = False)
            if decomp_cost < 1e-5:
                count += 1
    #         planted_H = feru.get_ferm_op(cur_tbt, True)
    #         print(feru.normal_ordered(planted_H))
        print(count)
        break

9
(316,)
[[0, 1], [2, 3], [4, 5, 6, 7]]


KeyboardInterrupt: 

In [49]:
#         print(feru.normal_ordered(planted_H))
print(count)

49


In [2]:
from multiprocessing import Pool
import time
import math
import sys
sys.path.append("../")
import ferm_utils as feru


if __name__ == "__main__":
    start_time = time.perf_counter()
    with Pool() as pool:
      r = pool.starmap(feru.add, [(1, 1), (2, 1), (3, 1)])
    print(r)
    finish_time = time.perf_counter()
    print("Program finished in {} seconds - using multiprocessing".format(finish_time-start_time))

[2, 3, 4]
Program finished in 4.037611500127241 seconds - using multiprocessing


In [11]:
"""Perform CAS-CAS Decomposition
Usage: python cas_run.py (mol) 
"""
import sys
sys.path.append("../")
import pickle
import io

import saveload_utils as sl
import ferm_utils as feru
import csa_utils as csau
import var_utils as varu
import openfermion as of
import numpy as np
import copy
import time 

### Parameters
mol = 'h2' if len(sys.argv) < 2 else sys.argv[1]
tol = 1e-5
save = False
method_name = 'CAS-CAS'

# Get two-body tensor
Hf = sl.load_fermionic_hamiltonian(mol)
_, gs = of.linalg.get_ground_state(of.linalg.get_sparse_operator(Hf))
spin_orb = of.count_qubits(Hf)  
spatial_orb = spin_orb // 2
Htbt = feru.get_chemist_tbt(Hf, spin_orb, spin_orb = True)
one_body = varu.get_one_body_correction_from_tbt(Hf, feru.get_chemist_tbt(Hf))
# feru.get_one_body_terms(Hf)
# varu.get_one_body_correction_from_tbt(Hf, Htbt)


onebody_matrix = feru.get_obt(one_body, n = spin_orb, spin_orb = True)
onebody_tbt = feru.onebody_to_twobody(onebody_matrix)
# print(onebody_tbt.shape)
# print(Htbt.shape)

Htbt = np.add(Htbt, onebody_tbt)
recombined = feru.get_ferm_op(Htbt, True)
print("Initial Norm: {}".format(np.sum(Htbt * Htbt)))

def partitions(n: int):
    """
    Return the all possible partition of the number n.
    """
    # base case of recursion: zero is the sum of the empty list
    if n == 0:
        yield []
        return
    
    # modify partitions of n-1 to form partitions of n
    for p in partitions(n-1):
        yield [1] + p
        if p and (len(p) < 2 or p[1] > p[0]):
            yield [p[0] + 1] + p[1:]

def partition_to_orbitals(partition: list[int]) -> list:
    """
    Return a orbital partitions from number partition of the total number of spin orbitals
    >>> partition_to_orbitals([2,2])
    [[0, 1], [2, 3]]
    >>> partition_to_orbitals([3,2,2])
    [[0, 1, 2], [3, 4], [5, 6]]
    """
    lis = [list(range(0+sum(partition[:i]),partition[i]+sum(partition[:i]))) for i in range(len(partition))]
    return lis

def valid_orbital_partitions(n: int) -> list[list[int]]:
    """
    Return the valid CAS orbital partitions with the number of spin orbitals n. 
    A partition is valid if it has more than one block and each block has at least 2 orbitals
    """     
    valid_partition = [i for i in list(partitions(n)) if min(i) > 1 and max(i) < n]
    valid_orbitals =[partition_to_orbitals(i) for i in valid_partition]
    return valid_orbitals
    
title = ["Partition","Norm of tbt", "relative tbt-norm", "Variance", "2-norm of eigenvalue spectrum"]
result = [title]
Hamiltonians = {}

with open("./planted_solution/" + mol + ".pkl", "wb") as f:
    pickle.dump(result, f)
# Hamiltonians[str(k)] = sol.x
with open("./planted_solution/" + mol + " Hamiltonians.pkl", "wb") as f:
    pickle.dump(Hamiltonians, f)    


from multiprocessing import Pool

with Pool() as pool:
  pool.starmap(feru.compute_cas_fragment, [(Htbt, k) for k in valid_orbital_partitions(spin_orb)])
# print(r)
# finish_time = time.perf_counter()
# print("Program finished in {} seconds - using multiprocessing".format(finish_time-start_time))

FileNotFoundError: [Errno 2] No such file or directory: '../ham_lib/-f_fer.bin'